In [2]:
import numpy as np
import pandas as pd 
from keras.utils import image_dataset_from_directory

In [3]:
df="/kaggle/input/glasses-or-no-glasses/faces-spring-2020"

In [4]:
train_data=image_dataset_from_directory(
df,
image_size=(256,256),
batch_size=32,
label_mode=None
)

Found 5000 files.


# # GAN

In [5]:
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout,Input,Conv2DTranspose,Reshape,ReLU,BatchNormalization
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [6]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [7]:
def build_gen():
    inputs=Input(shape=(100,))
    D=Dense(16*16*256)(inputs)
    D=Reshape((16,16,256))(D)
    C=Conv2DTranspose(256,(3,3),padding='same',strides=(2,2),kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
    C=ReLU()(C)
    C=BatchNormalization()(C)
    C=Dropout(0.5)(C)
    C=Conv2DTranspose(128,(3,3),padding='same',strides=(2,2),kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=BatchNormalization()(C)
    C=Dropout(0.5)(C)
    C=Conv2DTranspose(64,(3,3),padding='same',strides=(2,2),kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=BatchNormalization()(C)
    C=Dropout(0.5)(C)
    C=Conv2DTranspose(32,(3,3),padding='same',strides=(2,2),kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=BatchNormalization()(C)
    C=Dropout(0.5)(C)
    outputs=Conv2D(3,(3,3),activation="sigmoid",padding="same")(C)
    model=Model(inputs,outputs)
    return model
gen=build_gen()
gen.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 65536)          │     6,619,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 32, 32, 256)    │       590,080 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 64, 64, 128)    │       295,040 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 128, 128, 64)   │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 256, 256, 32)   │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 3)    │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,599,299 (28.99 MB)

 Trainable params: 7,598,339 (28.99 MB)

 Non-trainable params: 960 (3.75 KB)

In [8]:
def build_dis():
    inputs=Input(shape=(256,256,3))
    C=Conv2D(32,(3,3),padding='same',strides=(2,2),kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
    C=ReLU()(C)
    C=BatchNormalization()(C)
    C=Dropout(0.5)(C)
    C=Conv2D(64,(3,3),padding='same',strides=(2,2),kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=BatchNormalization()(C)
    C=Dropout(0.5)(C)
    C=Conv2D(128,(3,3),padding='same',strides=(2,2),kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=BatchNormalization()(C)
    C=Dropout(0.5)(C)
    C=Conv2D(256,(3,3),padding='same',strides=(2,2),kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=BatchNormalization()(C)
    C=Dropout(0.5)(C)
    C=Flatten()(C)
    outputs=Dense(1)(C)
    model=Model(inputs,outputs)
    return model
dis=build_dis()
dis.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_4 (ReLU)                  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_5 (ReLU)                  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_6 (ReLU)                  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_7 (ReLU)                  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │        65,537 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 455,873 (1.74 MB)

 Trainable params: 454,913 (1.74 MB)

 Non-trainable params: 960 (3.75 KB)

In [9]:
def build_gan(generator,discriminator):
    discriminator.trainable=False
#     discriminator.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')
    inputs=Input(shape=(100,))
    fake_img=generator(inputs)
    outputs=discriminator(fake_img)
    model=Model(inputs,outputs)
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

    return model
gan=build_gan(gen,dis)
gan.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_1 (Functional)       │ (None, 256, 256, 3)    │     7,599,299 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_3 (Functional)       │ (None, 1)              │       455,873 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,055,172 (30.73 MB)

 Trainable params: 7,598,339 (28.99 MB)

 Non-trainable params: 456,833 (1.74 MB)

In [ ]:
def train_gan(gan,gen,dis,epochs,dataset):
    dis.trainable=True
    dis.compile(optimizer='adam',loss='mse',metrics=['accuracy'])
    gan.compile(optimizer='adam',loss='mse',metrics=['accuracy'])
    for epoch in range(epochs):
        for real_img in dataset:
            real_label=np.ones((32,8))
            fake_label=np.zeros((32,8))
            noise=np.random.normal(0,1,(32,100))
            fake_img=gen(noise)
            dis.trainable=True
            dis.train_on_batch(real_img,real_label)
            dis.train_on_batch(fake_img,fake_label)
            dis.trainable=False
            gg=gan.train_on_batch(noise,real_label)
        print(f"Epoch {epoch + 1}/{epochs} completed")
        break
    return gg
model=train_gan(gan,gen,dis,5,train_data)


In [ ]:
noise=np.random.normal(0,1,(32,100))
model.predict(noise)